# Recomendacao de usuarios compativeis para participar de um projeto

      A descricao do projeto é dada pela lista de tecnologias/linguagens =['JavaScript','JavaScript','Python','JavaScript','JavaScript','C++','JavaScript','Lua','C']

    Você pode repetir uma tecnologias quantas vzs quiser, isso aumenta o peso da tecnologia especifica, e ajuda a encontrar usuarios mais especialistas.




In [26]:
%%time

import pandas as pd
import numpy as np
import re

from gensim.corpora import Dictionary
from gensim.models import TfidfModel
from gensim.models import Word2Vec
from multiprocessing import cpu_count
from gensim.similarities import MatrixSimilarity, SparseMatrixSimilarity, SoftCosineSimilarity

# import logging
# logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)


CPU times: user 20 µs, sys: 0 ns, total: 20 µs
Wall time: 23.6 µs


In [27]:
users_repositories = pd.read_csv('../data/users__repositories.csv', dtype=str)
users_repositories.head()

,User,Repo_Member,mRepo_Language,Repo_Owner,oRepo_Language
0,tarruda,libmpack/libmpack,C,"tarruda/Algoritmos,tarruda/archdb,tarruda/back...","JavaScript,JavaScript,JavaScript,JavaScript,Py..."
1,mairatma,"deprecate/steel-avatar,deprecate/steel-cell-de...","CSS,CSS,HTML,CSS,HTML,CSS,JavaScript,CSS,JavaS...","mairatma/alloy-ui,mairatma/alloyui.com,mairatm...","JavaScript,JavaScript,TypeScript,JavaScript,Ja..."
2,joselitojunior1,"acmh/maecoruja,Cisneiros/projeto-anfa,demianbo...","JavaScript,Java,Java,HTML,JavaScript,None","joselitojunior1/abigobaldo-nunes-adventures,jo...","JavaScript,HTML,CSS,None,None,None,None,GCC Ma..."
3,marcelcaraciolo,"irgmedeiros/TCCRecommender,jg1141/Open-Allure-DS","Python,Python","marcelcaraciolo/apontador-api-libs,marcelcarac...","PHP,Python,Python,None,Python,Python,Python,No..."
4,luanfonceca,"andrezap/analise_expressao_genica,andrezap/arv...","Java,Java,Java,Ruby,Ruby,JavaScript,JavaScript...","luanfonceca/168horas,luanfonceca/4stoq,luanfon...","CSS,Ruby,Python,JavaScript,Python,Python,None,..."


In [28]:
users_repositories.describe()

,User,Repo_Member,mRepo_Language,Repo_Owner,oRepo_Language
count,1992,785,785,1630,1630
unique,1981,764,413,1622,1207
top,khurama,consensotec/gsan,Java,"diogo-lins/BankAccount,diogo-lins/com-liferay-...",None
freq,2,3,78,2,132


In [29]:
# Filtrando e Transformando o dataframe para dados de interesse

repos_langs = pd.DataFrame({ 
    'User': users_repositories['User'],
    'Repos': (users_repositories['Repo_Member']+','+users_repositories['Repo_Owner']).fillna(''),
    'Languages': (users_repositories['mRepo_Language']+','+users_repositories['oRepo_Language']).fillna('')})
repos_langs.head()

,User,Repos,Languages
0,tarruda,"libmpack/libmpack,tarruda/Algoritmos,tarruda/a...","C,JavaScript,JavaScript,JavaScript,JavaScript,..."
1,mairatma,"deprecate/steel-avatar,deprecate/steel-cell-de...","CSS,CSS,HTML,CSS,HTML,CSS,JavaScript,CSS,JavaS..."
2,joselitojunior1,"acmh/maecoruja,Cisneiros/projeto-anfa,demianbo...","JavaScript,Java,Java,HTML,JavaScript,None,Java..."
3,marcelcaraciolo,"irgmedeiros/TCCRecommender,jg1141/Open-Allure-...","Python,Python,PHP,Python,Python,None,Python,Py..."
4,luanfonceca,"andrezap/analise_expressao_genica,andrezap/arv...","Java,Java,Java,Ruby,Ruby,JavaScript,JavaScript..."


## Construindo o dicionario: frequencia das palavras

### Languages

In [30]:
# Selecao de caracteristicas: vetor de linguagens
languages = repos_langs['Languages'].apply(lambda x: str(x).split(','))
languages.head()

0    [C, JavaScript, JavaScript, JavaScript, JavaSc...
1    [CSS, CSS, HTML, CSS, HTML, CSS, JavaScript, C...
2    [JavaScript, Java, Java, HTML, JavaScript, Non...
3    [Python, Python, PHP, Python, Python, None, Py...
4    [Java, Java, Java, Ruby, Ruby, JavaScript, Jav...
Name: Languages, dtype: object

### Repos

In [31]:
repos = repos_langs['Repos']\
            .apply(lambda x: str(x).split(','))\
            .apply(lambda x: np.concatenate([r.split('/') for r in x]))
repos.head()

0    [libmpack, libmpack, tarruda, Algoritmos, tarr...
1    [deprecate, steel-avatar, deprecate, steel-cel...
2    [acmh, maecoruja, Cisneiros, projeto-anfa, dem...
3    [irgmedeiros, TCCRecommender, jg1141, Open-All...
4    [andrezap, analise_expressao_genica, andrezap,...
Name: Repos, dtype: object

In [32]:
#  building dictionary based on languages dataset
dictionary = Dictionary(languages)
print("Number of unique words: %d" % len(dictionary))
print(dictionary)

Number of unique words: 102
Dictionary(102 unique tokens: ['C', 'C++', 'CMake', 'CoffeeScript', 'HTML']...)


## Train Model for predicions(similaridade): tf-idf

In [33]:
#  Traing Model for predicions(similaridade): tf-idf
tfidf = TfidfModel(dictionary=dictionary)

# print(dictionary.token2id)

In [34]:
# Novo Dado a ser rankeado
new_doc = ['JavaScript','JavaScript','Python','JavaScript','JavaScript','C++','JavaScript','Lua','C'] # novo exemplo

# Transformando novo exemplo to vetor (lang_idx, freq)
new_vec = dictionary.doc2bow(new_doc) # frequency vector
new_vec

[(0, 1), (1, 1), (7, 5), (8, 1), (12, 1)]

In [35]:
# ranking dos termos presentes em new_vec utilizando o mode tfidf
print(tfidf[new_vec]) # vector score

[(0, 0.2544226420392753), (1, 0.2679520076775512), (7, 0.7610542410669464), (8, 0.4989679145224941), (12, 0.18787842672965765)]


## Construindo a Matriz de similaridades

In [36]:
# Transforming dataset values
# vetor de frequencia de palavras = languages freq.
bow_corpus = [dictionary.doc2bow(text) for text in list(languages)] #frequency vector for all languages dataset

# Tecnica: Calculo de similaridade com Cosin
index = SparseMatrixSimilarity(tfidf[bow_corpus], num_features=102)

# LISTA DE TOP 10 USUARIOS RECOMENDADOS para new_vec

In [37]:
similarities = index[tfidf[new_vec]]
recomendations = list(enumerate(similarities))
top_recs = sorted(recomendations, key=lambda rating: rating[1], reverse=True)
repos_langs.iloc[[tr[0] for tr in top_recs[:10]], 0]

638           AleffSouza
1075       reueljonathan
0                tarruda
580         felipemobile
197           leoreinaux
992       marcuscarneiro
133     FranciscoMarinho
1423         bureauworks
1793               EFTBR
160                 acmh
Name: User, dtype: object

#End.

## Extra: Resultados com o Algoritmo sofcosin

In [39]:
# Algoritmo: Calculo de similaridade com SofCosin

w2v_model = Word2Vec(languages, workers=cpu_count(), size=102, seed=12345)
similarity_matrix = w2v_model.wv.similarity_matrix(dictionary, tfidf, nonzero_limit=100)

index = SoftCosineSimilarity( tfidf[bow_corpus], similarity_matrix)

similarities = index[tfidf[new_vec]]
recs_soft = list(enumerate(similarities))
# print(recs_soft)

top_recs_soft = sorted(recs_soft, key=lambda rating: rating[1], reverse=True)
repos_langs.iloc[[tr[0] for tr in top_recs_soft[:10]], 0]

638        AleffSouza
457         ntocampos
146     foolOnTheHill
1075    reueljonathan
138        PedroRossi
197        leoreinaux
160              acmh
412              Rom7
665      MaryCaroline
105        casevictor
Name: User, dtype: object